In [ ]:
from manim import *
import random

from manim.utils.color.AS2700 import Y14_GOLDEN_YELLOW

class PROPRO(Scene):
    def construct(self):
        random.seed(42)

        t1 = Text("Boosting", font="Microsoft YaHei", font_size=110, color=GOLD_A, weight="BOLD", z_index=1)
        t1.move_to(UP*4).set_opacity(0)

        self.play(t1.animate.shift(DOWN*4).set_opacity(1))
        self.wait(0.5)
        self.play(t1.animate.shift(DOWN*5).set_opacity(0))

        # 表格总体尺寸
        total_width = 12.0
        total_height = 0.6
        cols = 20
        # 外框矩形
        outer = Rectangle(
            width=total_width,
            height=total_height,
            stroke_color=WHITE,
            stroke_width=2,
            fill_opacity=0,
        )
        outer.to_edge(UP, buff=0.5)
        # 每个小方格
        col_w = total_width / cols
        squares = VGroup()
        left_x = outer.get_left()[0]
        bottom_y = outer.get_bottom()[1]
        for i in range(cols):
            sq = Rectangle(
                width=col_w,
                height=total_height,
                stroke_color=WHITE,
                stroke_width=1,
                fill_opacity=0
            )
            sq.move_to(np.array([left_x + (i + 0.5) * col_w, bottom_y + total_height / 2, 0]))
            squares.add(sq)
        # 动画：先画外框和格子
        self.play(Create(outer))
        self.play(LaggedStart(*[Create(s) for s in squares], lag_ratio=0.02))
        self.wait(3)

        # ---------- 所有方格变红 ----------
        self.play(*[sq.animate.set_fill(WHITE, opacity=0.5) for sq in squares])
        self.wait(1)
        # ---------- 每个方格内写 ωi ----------
        labels = VGroup()
        for i, sq in enumerate(squares, start=1):
            wi = MathTex(r"\boldsymbol{\omega_1}", font_size=40, color=WHITE)
            wi.move_to(sq.get_center()).shift(DOWN*0.8)
            labels.add(wi)
        self.play(Write(labels))
        self.wait(1)
        # --------- 箭头 + 矩形 ----------
        arrow1 = Arrow(
            start=outer.get_bottom()+DOWN*0.8, 
            end=outer.get_bottom() + DOWN*1.5, 
            buff=0, 
            color=WHITE
        )
        learner1 = Rectangle(
            width=3.5, height=1.2, 
            stroke_color=RED, stroke_width=3, fill_color=RED, fill_opacity=0.2
        )
        learner1.next_to(arrow1, DOWN, buff=0.3)
        text1 = Text("基学习器1", font="Microsoft YaHei", font_size=48, color=RED, weight="BOLD")
        text1.move_to(learner1.get_center())
        self.play(Create(arrow1), Create(learner1), Write(text1))
        self.wait(1)

        # ---------- 矩形下面的箭头 ----------
        arrow2 = Arrow(
            start=learner1.get_bottom() + DOWN*0.1,
            end=learner1.get_bottom() + DOWN*1,
            buff=0,
            color=WHITE
        )
        self.play(Create(arrow2))
        self.wait(0.5)
        # ---------- 在矩形下面画一个和上面一样的表格 ----------
        # 新表格位置参考 arrow2 终点
        outer2 = outer.copy()
        outer2.next_to(arrow2, DOWN, buff=0.3)

        squares2 = VGroup()
        col_w2 = total_width / cols
        left_x2 = outer2.get_left()[0]
        bottom_y2 = outer2.get_bottom()[1]
        for i in range(cols):
            sq2 = Rectangle(
                width=col_w2,
                height=total_height,
                stroke_color=WHITE,
                stroke_width=1,
                fill_opacity=0
            )
            sq2.move_to(np.array([left_x2 + (i + 0.5) * col_w2, bottom_y2 + total_height / 2, 0]))
            squares2.add(sq2)
        # ---------- 随机14个绿色 + 6个红色 ----------
        indices = list(range(cols))
        random.shuffle(indices)
        green_indices = indices[:14]
        red_indices = indices[14:]

        green_squares = VGroup(*[squares2[i] for i in green_indices])
        red_squares = VGroup(*[squares2[i] for i in red_indices])


        # 方格默认半透明红色填充
        self.play(Create(outer2))
        self.play(LaggedStart(*[Create(s) for s in squares2], lag_ratio=0.02))
        self.play(
            *[sq.animate.set_fill(GREEN_C, opacity=1) for sq in green_squares],
            *[sq.animate.set_fill(RED_C, opacity=1) for sq in red_squares]
        )
        self.wait(1)
        # ---------- 新表格每个方格内写 ωi ----------
        labels2 = VGroup()
        for i, sq in enumerate(squares2, start=1):
            wi2 = MathTex(r"\boldsymbol{\omega_1}", font_size=40, color=WHITE)
            wi2.move_to(sq.get_center()).shift(DOWN*0.8)
            labels2.add(wi2)
        self.play(Write(labels2))
        self.wait(1)
        # ---------- 先更新绿色方格 ----------
        green_animations = []
        for sq, wi2 in zip(squares2, labels2):
            if sq.get_fill_color() in [GREEN_C, GREEN]:
                green_animations.append(
                    Transform(
                        wi2,
                        MathTex(r"\boldsymbol{\omega_1}", font_size=40, color=GREEN_C).move_to(sq.get_center()).shift(DOWN*0.8)
                    )
                )
        self.play(*green_animations)
        self.wait(1.5)
        # ---------- 再更新红色方格 ----------
        red_animations = []
        for sq, wi2 in zip(squares2, labels2):
            if sq.get_fill_color() in [RED_C, RED]:
                red_animations.append(
                    Transform(
                        wi2,
                        MathTex(r"\boldsymbol{\omega_2}", font_size=40, color=RED_C).move_to(sq.get_center()).shift(DOWN*0.8)
                    )
                )
        self.play(*red_animations)
        self.wait(2)
        gg1 = VGroup(
            outer, squares, labels, 
            arrow1, learner1, text1, 
            arrow2
        )
        gg2 = VGroup(outer2, squares2, labels2)
        self.play(
            gg1.animate.shift(UP*7),
            gg2.animate.shift(UP*4.6)
        )
        self.wait(2)

        # ---------- 新表格下面画箭头 ----------
        arrow3 = Arrow(
            start=outer2.get_bottom() + DOWN*0.2 + DOWN*0.6,
            end=outer2.get_bottom() + DOWN*1+DOWN*0.6,
            buff=0,
            color=WHITE
        )
        self.play(Create(arrow3))
        self.wait(0.5)
        # ---------- 基学习器2矩形 ----------
        learner2 = Rectangle(
            width=3.5, height=1.2,
            stroke_color=GREEN, stroke_width=3,
            fill_color=GREEN, fill_opacity=0.2
        )
        learner2.next_to(arrow3, DOWN, buff=0.3)
        text2 = Text("基学习器2", font="Microsoft YaHei", font_size=48, color=GREEN, weight="BOLD")
        text2.move_to(learner2.get_center())
        self.play(Create(learner2), Write(text2))
        self.wait(1)
        # ---------- 矩形下面再画箭头 ----------
        arrow4 = Arrow(
            start=learner2.get_bottom() + DOWN*0.1,
            end=learner2.get_bottom() + DOWN*1,
            buff=0,
            color=WHITE
        )
        self.play(Create(arrow4))
        self.wait(2)
        # ---------- 新表格位置参考 arrow4 ----------
        outer3 = outer2.copy()
        outer3.next_to(arrow4, DOWN, buff=0.3)

        squares3 = VGroup()
        col_w3 = total_width / cols
        left_x3 = outer3.get_left()[0]
        bottom_y3 = outer3.get_bottom()[1]
        for i in range(cols):
            sq3 = Rectangle(
                width=col_w3,
                height=total_height,
                stroke_color=WHITE,
                stroke_width=1,
                fill_opacity=0
            )
            sq3.move_to(np.array([left_x3 + (i + 0.5) * col_w3, bottom_y3 + total_height / 2, 0]))
            squares3.add(sq3)
            
        random.seed(42)  # 可选，保证重复效果
        red_indices_gg2 = [i for i in range(cols) if i in red_indices]  # 上一次红色方格索引
        random.shuffle(red_indices_gg2)
        red3_indices = red_indices_gg2[:3]  # 选 3 个显示为红色
        
        self.play(Create(outer3))
        self.play(LaggedStart(*[Create(s) for s in squares3], lag_ratio=0.02))
        self.play(
            *[sq.animate.set_fill(GREEN_C, opacity=1) for sq in squares3],
            *[squares3[i].animate.set_fill(RED_C, opacity=1) for i in red3_indices]
        )
        self.wait(1)
        # ---------- 在新表格下再写 labels2 ----------
        labels22 = labels2.copy()
        labels22.next_to(outer3, DOWN).shift(DOWN*0.2)
        self.play(Write(labels22))
        self.wait(2)
        # ---------- 更新 labels22 中红色方格对应的标签 ----------
        final_animations = []
        for i, sq in enumerate(squares3):
            label = labels22[i]
            if sq.get_fill_color() in [RED_C, RED]:  # 如果方格是红色
                new_label = MathTex(r"\boldsymbol{\omega_3}", font_size=40, color=Y14_GOLDEN_YELLOW)
                new_label.move_to(label.get_center())
                final_animations.append(Transform(label, new_label))

        self.play(*final_animations)
        self.wait(2)
        # ---------- 将最后的新表格组合 ----------
        gg3 = VGroup(outer3, squares3, labels22)
        gg4 = VGroup(gg2, arrow3, arrow4, learner2, text2)
        # ---------- 整体上移 ----------
        self.play(
            gg4.animate.shift(UP*7),     
            gg3.animate.shift(UP*4.6)      # 最后的新表格整体上移
        )
        self.wait(2)
        # ---------- 在 gg3 下方画箭头 ----------
        arrow5 = Arrow(
            start=outer3.get_bottom() + DOWN*0.2 + DOWN*0.6,
            end=outer3.get_bottom() + DOWN*1 + DOWN*0.6,
            buff=0,
            color=WHITE
        )
        self.play(Create(arrow5))
        self.wait(0.5)
        # ---------- 基学习器3矩形 ----------
        learner3 = Rectangle(
            width=3.5, height=1.2,
            stroke_color=Y14_GOLDEN_YELLOW, stroke_width=3,
            fill_color=Y14_GOLDEN_YELLOW, fill_opacity=0.2
        )
        learner3.next_to(arrow5, DOWN, buff=0.3)
        text3 = Text("基学习器3", font="Microsoft YaHei", font_size=48, color=Y14_GOLDEN_YELLOW, weight="BOLD")
        text3.move_to(learner3.get_center())
        self.play(Create(learner3), Write(text3))
        self.wait(1)
        # ---------- 矩形下面再画箭头 ----------
        arrow6 = Arrow(
            start=learner3.get_bottom() + DOWN*0.1,
            end=learner3.get_bottom() + DOWN*1,
            buff=0,
            color=WHITE
        )
        self.play(Create(arrow6))
        self.wait(1)
        t2 = Text(".......................", font="Microsoft YaHei", font_size=110, color=GOLD_A, weight="BOLD")
        t2.next_to(arrow6, DOWN, buff=0.5)
        self.play(FadeIn(t2))
        self.wait(2)
        gg5 = VGroup(gg3, arrow5, arrow6, learner3, text3)
        gg6 = VGroup(t2)
        self.play(
            gg5.animate.shift(UP*7),     
            gg6.animate.shift(UP*4.6),
        )
        # ---------- 在 t2 下方画箭头 ----------
        arrow7 = Arrow(
            start=t2.get_bottom() + DOWN*0.7,
            end=t2.get_bottom() + DOWN*1.5,
            buff=0,
            color=WHITE
        )
        self.play(Create(arrow7))
        self.wait(0.5)

        vector_tex = f"[\\omega_1, \\omega_2, \\omega_3, \\dots, \\omega_n]"
        vec = MathTex(vector_tex, font_size=60, color=WHITE)
        vec.move_to(ORIGIN).scale(1.3).shift(DOWN*0.25)
        self.play(Write(vec))
        self.wait(2)

        t3 = Text("加权投票 / 加权平均", font="Microsoft YaHei", font_size=60, color=GOLD_A, weight="BOLD")
        t3.move_to(ORIGIN).shift(DOWN*1.75)
        self.play(Write(t3))
        self.wait(2)

        gray_screen = Rectangle(
            width=self.camera.frame_width,
            height=self.camera.frame_height,
            fill_color=DARK_GRAY,
            fill_opacity=0.98,
            stroke_width=0,
            z_index=0
        )
        t1.move_to(UP*4).set_opacity(0)
        self.play(
            FadeIn(gray_screen),
            t1.animate.shift(DOWN*2).set_opacity(1),
        )
        self.wait(1)
        steps = [
            "1. 初始：给所有样本相同的权重",
            "2. 第一个模型训练 → 预测 → 看错了哪些样本",
            "3. 提高错分样本的权重（让下一个模型更关注错误）",
            "4. 新模型训练 → 融合集成（加权投票 / 加权平均）",
            "5. 不断迭代，直到误差足够小或达到设定轮数"
        ]
        step_texts = VGroup(*[
            Text(step, font="Microsoft YaHei", font_size=36, color=WHITE)
            for step in steps
        ])
        # 垂直排列
        step_texts.arrange(DOWN, aligned_edge=LEFT, buff=0.5)
        step_texts.to_edge(DOWN, buff=0.5).shift(RIGHT*0.5)
        self.play(FadeIn(step_texts, shift=UP))
        self.wait(3)

        t33 = Text("逐步纠错+模型叠加", font="Microsoft YaHei", font_size=65, color=GOLD_B, weight="BOLD")
        t33.move_to(ORIGIN).shift(DOWN*0.5)
        t4 = Text("系统误差↓", font="Microsoft YaHei", font_size=65, color=GOLD_C, weight="BOLD")
        t4.move_to(ORIGIN).shift(DOWN*0.5)
        t5 = Text("偏差↓", font="Microsoft YaHei", font_size=65, color=GOLD_D, weight="BOLD")
        t5.move_to(ORIGIN).shift(DOWN*0.5)
        t6 = Text("欠拟合↓", font="Microsoft YaHei", font_size=65, color=GOLD_E, weight="BOLD")
        t6.move_to(ORIGIN).shift(DOWN*0.5)
        t7 = Text("过拟合↑", font="Microsoft YaHei", font_size=65, color=RED_C, weight="BOLD")
        t7.move_to(ORIGIN).shift(DOWN*0.5+RIGHT*2.5)
        t8 = Text("串行", font="Microsoft YaHei", font_size=85, color=GOLD_D, weight="BOLD")
        t8.move_to(ORIGIN).shift(UP*0.3)
        t9 = Text("速度慢↓", font="Microsoft YaHei", font_size=65, color=RED_C, weight="BOLD")
        t9.next_to(t7, DOWN, buff=0.5).shift(DOWN*0.9)
        self.play(
            step_texts.animate.shift(DOWN*5),
            FadeIn(t33)
        )
        self.wait(2)
        self.play(Transform(t33, t4))
        self.wait(2)
        self.play(Transform(t33, t5))
        self.wait(2)
        self.play(Transform(t33, t6))
        self.wait(5)
        self.play(
            t33.animate.shift(LEFT*2.5).set_color(GREEN_C),
            FadeIn(t7)
        )
        self.wait(2)
        self.play(
            t33.animate.shift(DOWN*1),
            t7.animate.shift(DOWN*1),
            FadeIn(t8)
        )
        self.wait(2)
        self.play(Write(t9))
        self.wait(5)